p()

In [1]:
import numpy as np

In [2]:
PATH_TO_DATA = "naivebayes-21\\trg.csv"

In [3]:
from collections import defaultdict

In [4]:
list_data = []
class_freq = defaultdict(lambda: 0)

with open(PATH_TO_DATA) as csv_file:
    for line in csv_file:
        line_lst = line.split(',')
        line_lst[-1] = line_lst[-1].replace('\n', '')

        class_freq[line_lst[1]] += 1 

        list_data.append(line_lst)


full_csv_data = np.array(list_data)
class_freq.pop('class')


labels = class_freq.keys()
headers = full_csv_data[0]
data = np.asarray(full_csv_data[1:,1:])
print(data[:2])

[['B'
  '"the 4 202 353 bp genome of the alkaliphilic bacterium bacillus halodurans c-125 contains 4066 predicted protein coding sequences cdss 2141 527 of which have functional assignments 1182 29 of which are conserved cdss with unknown function and 743 18 3 of which have no match to any protein database among the total cdss 88 match sequences of proteins found only in bacillus subtilis and 667 are widely conserved in comparison with the proteins of various organisms including bsubtilis the b halodurans genome contains 112 transposase genes indicating that transposases have played an important evolutionary role in horizontal gene transfer and also in internal genetic rearrangement in the genome strain c-125 lacks some of the necessary genes for competence such as coms srfa and rapc supporting the fact that competence has not been demonstrated experimentally in c-125 there is no paralog of tupa encoding teichuronopeptide which contributes to alkaliphily in the c-125 genome and an orth

In [5]:
print(class_freq.items())
print(labels)

dict_items([('B', 1602), ('A', 128), ('E', 2144), ('V', 126)])
dict_keys(['B', 'A', 'E', 'V'])


In [6]:
from numpy.random import default_rng

ABSTRT_I = 1
LBL_I = 0

In [7]:
def train_test_split(X, y, test_size=None, stratify=False):

    rng = default_rng()
    arr = np.arange(10)
    arr1 = np.arange(10)

    rng.random(arr)
    print(arr)

In [8]:
train_test_split(None, None, None)

[0 1 2 3 4 5 6 7 8 9]


In [9]:
 def get_stratified_kfold_splits(data, k=10):
    instances_of_class = dict()

    for label in labels:
        i_arr = data[:, 0] == label
        instances_of_class[label] = data[i_arr]
    
    stratified_splits = dict()
    
    # Upsample to divisible by k
    for class_label, class_instances in instances_of_class.items():
        n = len(class_instances)
        upsample_amt = k - n % k

        random_indices = np.random.choice(class_instances[:, ABSTRT_I], size=upsample_amt, replace=False)
        random_indices.resize(random_indices.shape[0], 2)
        random_indices[:, -1] = class_label

        upsampled_class_instances = np.concatenate((class_instances, random_indices), 0)
        assert len(upsampled_class_instances) % k == 0, "num examples should be divisible by k"

        stratified_splits[class_label] = np.split(upsampled_class_instances, k)

    for i in range(k):
        kth_data_lst = [stratified_splits[label][i] for label in labels]
        
        kth_data = np.concatenate(kth_data_lst)
        np.random.shuffle(kth_data)

        yield kth_data

In [10]:
stratified_data = get_stratified_kfold_splits(data)

In [11]:
def get_word_counts(data):

    all_words = set(word for i in range(len(data)) for word in data[i][ABSTRT_I].split()) 

    all_words_i = {word:i for i, word in enumerate(all_words)}

    freq_mat = np.zeros((len(data), len(all_words)))

    for i in range(len(data)):
        for word in data[i][ABSTRT_I].split():
            freq_mat[i,all_words_i[word]] += 1

    return freq_mat

In [12]:
def get_sum_mat(freq_mat, data):
    sum_mat = np.zeros((len(labels), freq_mat.shape[1]))
    
    for i,lbl in enumerate(labels):
        sum_mat[i,:] = np.sum(freq_mat, axis=0, where=lbl in data[:,LBL_I]) 
    return sum_mat

In [13]:
sd = next(stratified_data)
X = sd[:-1]
y = sd[-1]

In [14]:
freq_mat = get_word_counts(sd)
summa = get_sum_mat(freq_mat, sd)
print(summa[0])
print(summa[1])
print(summa[2])


[ 1.  1.  5. ...  1. 54.  2.]
[ 1.  1.  5. ...  1. 54.  2.]
[ 1.  1.  5. ...  1. 54.  2.]


In [15]:
NUM_FEATURES = 10

